In [1]:
import os
import warnings

from datasets import load_dataset
from peft import get_peft_model
from peft import LoraConfig
from peft import TaskType
from transformers import DataCollatorForSeq2Seq
from transformers import EarlyStoppingCallback
from transformers import Trainer
from transformers import TrainingArguments

from tools.config import Config
from tools.constant import ModelMode
from tools.constant import ModelName
from tools.utils import File
from tools.utils import Time

/root/miniconda3/envs/qwen/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 這些警告已确认与用户使用无关，大多是Qwen的问题
warnings.filterwarnings("ignore",
                        message="Passing the following arguments to `Accelerator` is deprecated")  # transformers的使用问题
warnings.filterwarnings("ignore", message=".*the use_reentrant parameter should be passed explicitly")  # Qwen的使用问题
warnings.filterwarnings("ignore", message="Could not find a config file")  # 不懂为啥保存时需要和远程仓库校对词汇表

In [3]:
# ================================================================================================
# 模型部分

model_name = ModelName.QWEN_7B_CHAT
mode = ModelMode.TRAIN
tokenizer = Config.get_tokenizer(model_name, mode=mode)

tokenizer

QWenTokenizer(name_or_path='/mnt/workspace/Qwen-7B-Chat', vocab_size=151851, model_max_length=32768, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'pad_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	
}

In [4]:
model = Config.get_model(model_name, mode=mode)
lora_config = LoraConfig(
    # 不加的话最后的model是PeftModel，加了是PeftModelForCausalLM，看代码貌似没有什么关键改动，但是以防万一加上吧
    task_type=TaskType.CAUSAL_LM,

    # finetune
    # r=64,
    # lora_alpha=16,
    # lora_dropout=0.05,

    # demo
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["c_attn", "c_proj", "w1", "w2"]
)

model = get_peft_model(model, lora_config)
model.enable_input_require_grads()  # peft与gradient checkpoint同时用时必须加
model.print_trainable_parameters()

model

loading model Qwen-7B-Chat...


Loading checkpoint shards: 100%|██████████| 8/8 [01:27<00:00, 10.92s/it]


trainable params: 17,891,328 || all params: 7,739,215,872 || trainable%: 0.2312


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): QWenLMHeadModel(
      (transformer): QWenModel(
        (wte): Embedding(151936, 4096)
        (drop): Dropout(p=0.0, inplace=False)
        (rotary_emb): RotaryEmbedding()
        (h): ModuleList(
          (0-31): 32 x QWenBlock(
            (ln_1): RMSNorm()
            (attn): QWenAttention(
              (c_attn): lora.Linear(
                (base_layer): Linear(in_features=4096, out_features=12288, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=12288, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )

In [5]:
# ================================================================================================
# 数据部分


dataset_dir = "/mnt/workspace/intermediate/_nl2sql_dataset_10000"
dataset = load_dataset("csv", data_files={
    "train": f"{dataset_dir}/nl2sql_train_dataset.csv",  # 10000
    "validation": f"{dataset_dir}/nl2sql_validation_dataset.csv",  # 1000
    "test": f"{dataset_dir}/nl2sql_test_dataset.csv"  # 1000
})


def func(example):
    question = example["问题"]
    sql = example["SQL"]
    return tokenizer.make_finetune_inputs(question, sql)


dataset = dataset.map(func, remove_columns=dataset.column_names["train"], num_proc=os.cpu_count())
train_dataset = dataset["train"]  # 10000
validation_dataset = dataset["validation"]  # 1000
test_dataset = dataset["test"]  # 1000

dataset

Generating train split: 10000 examples [00:00, 99952.43 examples/s]
Generating validation split: 1000 examples [00:00, 93041.35 examples/s]
Generating test split: 1000 examples [00:00, 89719.65 examples/s]
Map (num_proc=8): 100%|██████████| 1000/1000 [00:00<00:00, 3897.56 examples/s]


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 10000
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1000
    })
})

In [6]:
# ================================================================================================
# 训练部分

# 按训练时间分文件夹，避免覆盖之前训练的模型
fmt = "%Y%m%d_%H%M%S"
output_dir = f"{Config.NL2SQL_FINETUNE_DIR}/{Time.current_time(fmt)}"
print(f"{output_dir=}")

training_args = TrainingArguments(
    output_dir=output_dir,
    seed=Config.SEED,

    # 训练
    num_train_epochs=10,  # 有early stop
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,

    # 验证
    evaluation_strategy="steps",
    eval_steps=50,
    per_device_eval_batch_size=16,

    # 日志
    # logging_strategy="steps",  # 默认就是这个
    logging_steps=50,  # 使用notebook时，如果开启了eval，只有进行eval的step才会打印日志

    # 保存
    # save_strategy="steps",  # 默认就是这个
    save_steps=50,  # 和eval保持一致
    save_total_limit=2,
    load_best_model_at_end=True,

    # 超参数
    learning_rate=3e-4,
    weight_decay=0.1,
    adam_beta2=0.95,
    warmup_ratio=0.01,
    lr_scheduler_type="cosine"
)

output_dir='/mnt/workspace/prepare/output/nl2sql_finetune/20240724_165829'


In [7]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer),
    callbacks=[EarlyStoppingCallback(early_stopping_patience=10)]
)

Detected kernel version 4.19.24, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [8]:
trainer.train()

You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.


Step,Training Loss,Validation Loss
50,0.495100,0.016890
100,0.013300,0.004692
150,0.004200,0.002930
200,0.004300,0.005339
250,0.003400,0.000735
300,0.003800,0.005010
350,0.003300,0.004904
400,0.002500,0.001813
450,0.003000,0.000617
500,0.004800,0.000806


TrainOutput(global_step=1450, training_loss=0.019985608052590798, metrics={'train_runtime': 6780.64, 'train_samples_per_second': 14.748, 'train_steps_per_second': 0.922, 'total_flos': 2.0241038728455782e+17, 'train_loss': 0.019985608052590798, 'epoch': 2.32})

In [9]:
trainer.save_model(f"{output_dir}/best")

In [10]:
# ================================================================================================
# 测试部分


trainer.evaluate(test_dataset)

{'eval_loss': 0.00038561440305784345,
 'eval_runtime': 71.3454,
 'eval_samples_per_second': 14.016,
 'eval_steps_per_second': 0.883,
 'epoch': 2.32}

In [11]:
# ================================================================================================
# ================================================================================================
# 直接执行B5的逻辑


db = Config.get_database()
answer_df = Config.get_sql_question_answer_df()

pred_df = answer_df.drop(columns=["问题聚类", "答案"])
question_num = len(pred_df)
questions = pred_df["问题"].tolist()
# true_sqls = pred_df["SQL"].tolist()
# true_results = pred_df["SQL结果"].tolist()

In [12]:
# =====================================================================================
# 预测sql

tokenizer.padding_side = "left"  # 批量推理必须为左填充
pred_sqls = model.batch(tokenizer, questions, batch_size=8)
pred_df["预测SQL"] = pred_sqls

pred_df["SQL正确"] = pred_df["SQL"] == pred_df["预测SQL"]
sql_correct_num = sum(pred_df["SQL正确"])
print(f"测试问题数：{question_num}")  # 600
print(f"SQL正确数：{sql_correct_num}")
print(f"SQL正确率：{sql_correct_num / question_num:.2%}")
# 展示bad case
pred_df.query("SQL正确 == False")

100%|██████████| 75/75 [12:49<00:00, 10.27s/it]

测试问题数：600
SQL正确数：572
SQL正确率：95.33%


,问题id,问题,SQL,SQL结果,预测SQL,SQL正确
1,2,请帮我查询出20210415日，建筑材料一级行业涨幅超过5%（不包含）的股票数量。,SELECT COUNT(DISTINCT(t1.股票代码)) AS 数量\nFROM A股票日行情表 t1 JOIN A股公司行业划分表 t2\nON t1.股票代码 = t2.股票代码\nAND t1.交易日 = t2.交易日期\nAND t1.交易日 = '20210415'\nAND t2.一级行业名称 = '建筑材料'\nAND (t1.[收盘价(元)] / t1.[昨收盘(元)] - 1) * 100 > 5\nLIMIT 1;,[{'数量': 2}],SELECT COUNT(DISTINCT(t1.股票代码) AS 数量\nFROM A股票日行情表 t1 JOIN A股公司行业划分表 t2\nON t1.股票代码 = t2.股票代码\nAND t1.交易日 = t2.交易日期\nAND t1.交易日 = '20210415'\nAND t2.一级行业名称 = '建筑材料'\nAND (t1.[收盘价(元)] / t1.[昨收盘(元)] - 1) * 100 > 5\nLIMIT 1;,False
5,7,2021年三季度，有多少家基金发生了净赎回?总共赎回了多少份额?记得给我四舍五入到小数点后两位哦。,"WITH t1 AS (\n SELECT *, 报告期期初基金总份额 - 报告期期末基金总份额 AS 净赎回\n FROM 基金规模变动表\n WHERE 截止日期 LIKE '2021-09-30%'\n)\nSELECT COUNT(*) AS 数量, ROUND(SUM(净赎回), 2) AS 净赎回份额\nFROM t1\nWHERE 净赎回 > 0\nLIMIT 1;","[{'数量': 2085, '净赎回份额': 317373703552.49}]","WITH t1 AS (\n SELECT *, 报告期期初基金总份额 - 报告期期末基金总份额 AS 净赎回\n FROM 基金规模变动表\n WHERE 截止日期 LIKE '2021-09-30%'\n)\nSELECT COUNT(*) AS 数量\nFROM t1\nWHERE 净赎回 > 0\nLIMIT 1;",False
13,20,在20201022，按照中信行业分类的行业划分标准，哪个一级行业的A股公司数量最多？,SELECT 一级行业名称\nFROM A股公司行业划分表\nWHERE 交易日期 = '20201022'\nAND 行业划分标准 = '中信行业分类'\nGROUP BY 一级行业名称\nORDER BY COUNT(*) DESC\nLIMIT 1;,[{'一级行业名称': '机械'}],SELECT 一级行业名称\nFROM A股公司行业划分表\nWHERE 行业划分标准 = '中信行业分类'\nGROUP BY 一级行业名称\nORDER BY COUNT(*) DESC\nLIMIT 1;,False
27,43,我想知道富兰克林国海大中华精选混合(QDII)(美元现汇)在2021年Q2的季报中，该基金的第1大重仓股的代码是什么?,SELECT 股票代码\nFROM 基金股票持仓明细\nWHERE 基金简称 = '富兰克林国海大中华精选混合(QDII)(美元现汇)'\nAND 持仓日期 = '20210630'\nAND 报告类型 = '季报'\nAND 第N大重仓股 = 1\nLIMIT 1;,[{'股票代码': '00700'}],SELECT 股票代码\nFROM 基金股票持仓明细\nWHERE 基金简称 = '富兰克林国海大中华精选混合(QDII)(美元现汇)\nAND 持仓日期 = '20210630'\nAND 报告类型 = '季报'\nAND 第N大重仓股 = 1\nLIMIT 1;,False
43,70,博时基金管理有限公司管理的股票型产品的数量有多少?,SELECT COUNT(基金代码) AS 数量\nFROM 基金基本信息\nWHERE 管理人 = '博时基金管理有限公司'\nAND 基金类型 = '股票型'\nLIMIT 1;,[{'数量': 16}],SELECT COUNT(*) AS 数量\nFROM 基金基本信息\nWHERE 管理人 = '博时基金管理有限公司'\nAND 基金类型 = '股票型'\nLIMIT 1;,False
61,101,请查询：在2020的年度报告中，个人投资者持有基金份额大于机构投资者持有基金份额的基金属于其他型类型的有几个。,SELECT COUNT(*) AS 数量\nFROM 基金基本信息\nWHERE 基金代码 IN (\n SELECT 基金代码\n FROM 基金份额持有人结构\n WHERE 定期报告所属年度 = 2020\n AND 报告类型 = '年度报告'\n AND 个人投资者持有的基金份额 > 机构投资者持有的基金份额\n)\nAND 基金类型 = '其他型'\nLIMIT 1;,[{'数量': 55}],SELECT COUNT(*) AS 数量\nFROM 基金基本信息\nWHERE 基金类型 = '其他型'\nLIMIT 1;,False
75,119,20190419日，请给出浦银安盛盛勤3个月定期开放债券C基金的管理人和累计单位净值。,"SELECT t1.管理人, t2.累计单位净值\nFROM 基金基本信息 t1 JOIN 基金日行情表 t2\nON t1.基金代码 = t2.基金代码\nAND t1.基金简称 = '浦银安盛盛勤3个月定期开放债券C'\nAND t2.交易日期 = '20190419'\nLIMIT 1;","[{'管理人': '浦银安盛基金管理有限公司', '累计单位净值': 1.0549}]","SELECT t1.管理人, ROUND(t2.累计单位净值, 2) AS 累计单位净值\nFROM 基金基本信息 t1 JOIN 基金日行情表 t2\nON t1.基金代码 = t2.基金代码\nAND t1.基金简称 = '浦银安盛盛勤3个月定期开放债券C'\nAND t2.交易日期 = '20190419'\nLIMIT 1;",False
122,203,20210106日，请给出汇安宜创量化精选混合C基金的管理人和累计单位净值。,"SELECT t1.管理人, t2.累计单位净值\nFROM 基金基本信息 t1 JOIN 基金日行情表 t2\nON t1.基金代码 = t2.基金代码\nAND t1.基金简称 = '汇安宜创量化精选混合C'\nAND t2.交易日期 = '20210106'\nLIMIT 1;","[{'管理人': '汇安基金管理有限责任公司', '累计单位净值': 1.5558}]","SELECT t1.管理人, ROUND(t1.累计单位净值, 2) AS 累计单位净值\nFROM 基金基本信息 t1 JOIN 基金日行情表 t2\nON t1.基金代码 = t2.基金代码\nAND t1.基金简称 = '汇安宜创量化精选混合C'\nAND t2.交易日期 = '20210106'\nLIMIT 1;",False
128,213,平安惠添纯债债券基金在20190930的季报里，前三大持仓占比的债券名称是什么?,SELECT 债券名称\nFROM 基金债券持仓明细\nWHERE 基金简称 = '平安惠添纯债债券'\nAND 持仓日期 = '20190930'\nAND 报告类型 = '季报'\nORDER BY 第N大重仓股 ASC\nLIMIT 3;,"[{'债券名称': '19进出04'}, {'债券名称': '国开1801'}, {'债券名称': '17铜官管廊债'}]",SELECT 债券名称\nFROM 基金债券持仓明细\nWHERE 持仓日期 = '20190930'\nAND 报告类型 = '季报'\nORDER BY 第N大重仓股 ASC\nLIMIT 3;,False
131,218,"我想了解一下红塔红土基金管理有限公司2019年成立的债券型基金,其管理费率的平均值是多少?请四舍五入保留小数点两位。","SELECT ROUND(AVG(管理费率), 2) AS 平均管理费率\nFROM 基金基本信息\nWHERE 管理人 = '红塔红土基金管理有限公司'\nAND 成立日期 LIKE '2019%'\nAND 基金类型 = '债券型'\nLIMIT 1;",[{'平均管理费率': 0.3}],"SELECT ROUND(AVG(管理费率), 2) AS 平均管理费率\nFROM 基金基本信息\nWHERE 基金类型 = '债券型'\nAND 成立日期 LIKE '2019%'\nLIMIT 1;",False


In [13]:
# =====================================================================================
# 执行sql


pred_results = [
    None if raw_result is None else str(raw_result.to_dict(orient="records"))
    for raw_result in db.batch_query(pred_sqls, raise_error=False)
]
pred_df["预测SQL结果"] = pred_results

pred_df["结果正确"] = pred_df["SQL结果"] == pred_df["预测SQL结果"]
execute_num = sum(pred_df["预测SQL结果"].notnull())
result_correct_num = sum(pred_df["结果正确"])
print(f"测试问题数：{question_num}")  # 600
print(f"成功执行数：{execute_num}")
print(f"成功执行率：{execute_num / question_num:.2%}")
print(f"结果正确数：{result_correct_num}")
print(f"结果正确率：{result_correct_num / question_num:.2%}")
print(f"结果正确/成功执行：{result_correct_num / execute_num:.2%}")
# 展示bad case
pred_df.query("结果正确 == False")

100%|██████████| 600/600 [05:37<00:00,  1.78it/s]

测试问题数：600
成功执行数：596
成功执行率：99.33%
结果正确数：577
结果正确率：96.17%
结果正确/成功执行：96.81%


,问题id,问题,SQL,SQL结果,预测SQL,SQL正确,预测SQL结果,结果正确
1,2,请帮我查询出20210415日，建筑材料一级行业涨幅超过5%（不包含）的股票数量。,SELECT COUNT(DISTINCT(t1.股票代码)) AS 数量\nFROM A股票日行情表 t1 JOIN A股公司行业划分表 t2\nON t1.股票代码 = t2.股票代码\nAND t1.交易日 = t2.交易日期\nAND t1.交易日 = '20210415'\nAND t2.一级行业名称 = '建筑材料'\nAND (t1.[收盘价(元)] / t1.[昨收盘(元)] - 1) * 100 > 5\nLIMIT 1;,[{'数量': 2}],SELECT COUNT(DISTINCT(t1.股票代码) AS 数量\nFROM A股票日行情表 t1 JOIN A股公司行业划分表 t2\nON t1.股票代码 = t2.股票代码\nAND t1.交易日 = t2.交易日期\nAND t1.交易日 = '20210415'\nAND t2.一级行业名称 = '建筑材料'\nAND (t1.[收盘价(元)] / t1.[昨收盘(元)] - 1) * 100 > 5\nLIMIT 1;,False,None,False
5,7,2021年三季度，有多少家基金发生了净赎回?总共赎回了多少份额?记得给我四舍五入到小数点后两位哦。,"WITH t1 AS (\n SELECT *, 报告期期初基金总份额 - 报告期期末基金总份额 AS 净赎回\n FROM 基金规模变动表\n WHERE 截止日期 LIKE '2021-09-30%'\n)\nSELECT COUNT(*) AS 数量, ROUND(SUM(净赎回), 2) AS 净赎回份额\nFROM t1\nWHERE 净赎回 > 0\nLIMIT 1;","[{'数量': 2085, '净赎回份额': 317373703552.49}]","WITH t1 AS (\n SELECT *, 报告期期初基金总份额 - 报告期期末基金总份额 AS 净赎回\n FROM 基金规模变动表\n WHERE 截止日期 LIKE '2021-09-30%'\n)\nSELECT COUNT(*) AS 数量\nFROM t1\nWHERE 净赎回 > 0\nLIMIT 1;",False,[{'数量': 2085}],False
27,43,我想知道富兰克林国海大中华精选混合(QDII)(美元现汇)在2021年Q2的季报中，该基金的第1大重仓股的代码是什么?,SELECT 股票代码\nFROM 基金股票持仓明细\nWHERE 基金简称 = '富兰克林国海大中华精选混合(QDII)(美元现汇)'\nAND 持仓日期 = '20210630'\nAND 报告类型 = '季报'\nAND 第N大重仓股 = 1\nLIMIT 1;,[{'股票代码': '00700'}],SELECT 股票代码\nFROM 基金股票持仓明细\nWHERE 基金简称 = '富兰克林国海大中华精选混合(QDII)(美元现汇)\nAND 持仓日期 = '20210630'\nAND 报告类型 = '季报'\nAND 第N大重仓股 = 1\nLIMIT 1;,False,None,False
61,101,请查询：在2020的年度报告中，个人投资者持有基金份额大于机构投资者持有基金份额的基金属于其他型类型的有几个。,SELECT COUNT(*) AS 数量\nFROM 基金基本信息\nWHERE 基金代码 IN (\n SELECT 基金代码\n FROM 基金份额持有人结构\n WHERE 定期报告所属年度 = 2020\n AND 报告类型 = '年度报告'\n AND 个人投资者持有的基金份额 > 机构投资者持有的基金份额\n)\nAND 基金类型 = '其他型'\nLIMIT 1;,[{'数量': 55}],SELECT COUNT(*) AS 数量\nFROM 基金基本信息\nWHERE 基金类型 = '其他型'\nLIMIT 1;,False,[{'数量': 115}],False
75,119,20190419日，请给出浦银安盛盛勤3个月定期开放债券C基金的管理人和累计单位净值。,"SELECT t1.管理人, t2.累计单位净值\nFROM 基金基本信息 t1 JOIN 基金日行情表 t2\nON t1.基金代码 = t2.基金代码\nAND t1.基金简称 = '浦银安盛盛勤3个月定期开放债券C'\nAND t2.交易日期 = '20190419'\nLIMIT 1;","[{'管理人': '浦银安盛基金管理有限公司', '累计单位净值': 1.0549}]","SELECT t1.管理人, ROUND(t2.累计单位净值, 2) AS 累计单位净值\nFROM 基金基本信息 t1 JOIN 基金日行情表 t2\nON t1.基金代码 = t2.基金代码\nAND t1.基金简称 = '浦银安盛盛勤3个月定期开放债券C'\nAND t2.交易日期 = '20190419'\nLIMIT 1;",False,"[{'管理人': '浦银安盛基金管理有限公司', '累计单位净值': 1.05}]",False
122,203,20210106日，请给出汇安宜创量化精选混合C基金的管理人和累计单位净值。,"SELECT t1.管理人, t2.累计单位净值\nFROM 基金基本信息 t1 JOIN 基金日行情表 t2\nON t1.基金代码 = t2.基金代码\nAND t1.基金简称 = '汇安宜创量化精选混合C'\nAND t2.交易日期 = '20210106'\nLIMIT 1;","[{'管理人': '汇安基金管理有限责任公司', '累计单位净值': 1.5558}]","SELECT t1.管理人, ROUND(t1.累计单位净值, 2) AS 累计单位净值\nFROM 基金基本信息 t1 JOIN 基金日行情表 t2\nON t1.基金代码 = t2.基金代码\nAND t1.基金简称 = '汇安宜创量化精选混合C'\nAND t2.交易日期 = '20210106'\nLIMIT 1;",False,None,False
128,213,平安惠添纯债债券基金在20190930的季报里，前三大持仓占比的债券名称是什么?,SELECT 债券名称\nFROM 基金债券持仓明细\nWHERE 基金简称 = '平安惠添纯债债券'\nAND 持仓日期 = '20190930'\nAND 报告类型 = '季报'\nORDER BY 第N大重仓股 ASC\nLIMIT 3;,"[{'债券名称': '19进出04'}, {'债券名称': '国开1801'}, {'债券名称': '17铜官管廊债'}]",SELECT 债券名称\nFROM 基金债券持仓明细\nWHERE 持仓日期 = '20190930'\nAND 报告类型 = '季报'\nORDER BY 第N大重仓股 ASC\nLIMIT 3;,False,"[{'债券名称': '16进出02'}, {'债券名称': '18远洋集团MTN002'}, {'债券名称': '18远洋集团MTN002'}]",False
131,218,"我想了解一下红塔红土基金管理有限公司2019年成立的债券型基金,其管理费率的平均值是多少?请四舍五入保留小数点两位。","SELECT ROUND(AVG(管理费率), 2) AS 平均管理费率\nFROM 基金基本信息\nWHERE 管理人 = '红塔红土基金管理有限公司'\nAND 成立日期 LIKE '2019%'\nAND 基金类型 = '债券型'\nLIMIT 1;",[{'平均管理费率': 0.3}],"SELECT ROUND(AVG(管理费率), 2) AS 平均管理费率\nFROM 基金基本信息\nWHERE 基金类型 = '债券型'\nAND 成立日期 LIKE '2019%'\nLIMIT 1;",False,[{'平均管理费率': 0.31}],False
134,223,"我想了解一下西部利得基金管理有限公司2020年成立的债券型基金,其管理费率的平均值是多少?请四舍五入保留小数点两位。","SELECT ROUND(AVG(管理费率), 2) AS 平均管理费率\nFROM 基金基本信息\nWHERE 管理人 = '西部利得基金管理有限公司'\nAND 成立日期 LIKE '2020%'\nAND 基金类型 = '债券型'\nLIMIT 1;",[{'平均管理费率': 0.44}],"SELECT ROUND(AVG(管理费率), 2) AS 平均管理费率\nFROM 基金基本信息\nWHERE 基金代码 IN (\n SELECT 基金代码\n FROM 基金规模变动表\n WHERE 基金代码 IN (\n SELECT 基金代码\n FROM 基金基本信息\n WHERE 管理人 = '西部利得基金管理有限公司'\n )\n AND 成立日期 LIKE '2020%'\n)\nLIMIT 1;",False,[{'平均管理费率': 0.76}],False
170,278,我想知道在20200331的季报里，中银证券价值精选灵活配置混合投资的股票分别是哪些申万一级行业？,SELECT DISTINCT(一级行业名称)\nFROM 

In [14]:
# =====================================================================================


# 保存下来，不要浪费
File.dataframe_to_csv(pred_df, f"{output_dir}/nl2sql_evaluate.csv")